<font color=purple>
    
Please note you need to run in the datalab enviroment in GCP, not locally.

Bring in your libraries

In [ ]:
import google.datalab.bigquery as bq
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import shutil

<font color=purple>
    
# Extract data from BigQuery 

There are several public big data sets available from google <a href="https://bigquery.cloud.google.com/publicdatasets/"> here.</a> 

The dataset that we will use is <a href="https://bigquery.cloud.google.com/table/nyc-tlc:yellow.trips"> the yellow taxi cabs of New York</a>. 

Write a SQL query to pick up the following fields
  * pickup_datetime,
  * pickup_longitude, pickup_latitude, 
  * dropoff_longitude, dropoff_latitude,
  * passenger_count,
  * trip_distance,
  * tolls_amount,
  * fare_amount,
  * total_amount

from the dataset and explore a small part of the data. 

We want a repeatable subset of the data so that if someone reruns this notebook, they will get the same results.

In [ ]:
rawdata = """
SELECT
  pickup_datetime,
  pickup_longitude, pickup_latitude, 
  dropoff_longitude, dropoff_latitude,
  passenger_count,
  trip_distance,
  tolls_amount,
  fare_amount,
  total_amount
FROM
  `nyc-tlc.yellow.trips`
WHERE
  MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),EVERY_N) = 1
"""

In [ ]:
query = rawdata.replace("EVERY_N", "1000")
print(query)
trips = bq.Query(query).execute().result().to_dataframe()
print("Total dataset is {} taxi rides".format(len(trips)))
trips[:10]

<font color=purple>
    
# Run a describe on the test data. Does this make ‘sense’ ?

* Can a Passenger count be < 1?
* Can fare amount be less than 0?
* If all of the taxi rides are in New York (40.7128° N, 74.0060° W),
    * what are good bounds on the longitude and latitude? 
        * (+/- 2°, +/- 5°, +/- 10°, +/- 100°?? )

This is added to your bigquery, after the where statment, and is re-run below:

AND passenger_count >= 1 
AND trip_distance > 0
AND fare_amount > 0
AND total_amount > 0
AND pickup_longitude < -72 AND pickup_longitude > -76
AND pickup_latitude > 38 AND pickup_latitude < 42
AND dropoff_longitude < -72 AND dropoff_longitude > -76
AND dropoff_latitude > 38 AND dropoff_latitude < 42

In [ ]:
trips.describe()

In [ ]:
plt.scatter(x = trips["trip_distance"], y = trips["fare_amount"])
plt.title('Taxi Trips')
plt.xlabel("Trip Distance")
plt.ylabel("Fare Amount")
plt.show()

In [ ]:
## the pick-ups
sns.lmplot(x='pickup_longitude', 
           y='pickup_latitude', 
           fit_reg=False, 
           data=trips);
## the drop offs
sns.lmplot(x='dropoff_longitude', 
           y='dropoff_latitude', 
           fit_reg=False, 
           data=trips);

In [ ]:
rawdata = """
SELECT
  pickup_datetime,
  pickup_longitude, pickup_latitude, 
  dropoff_longitude, dropoff_latitude,
  passenger_count,
  trip_distance,
  tolls_amount,
  fare_amount,
  total_amount
FROM
  `nyc-tlc.yellow.trips`
WHERE
  MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),EVERY_N) = 1
  AND passenger_count >= 1 AND trip_distance > 0 
  AND fare_amount > 0 AND total_amount > 0 
  AND pickup_longitude < -72 AND pickup_longitude > -76 
  AND pickup_latitude > 38 AND pickup_latitude < 42
  AND dropoff_longitude < -72 AND dropoff_longitude > -76 
  AND dropoff_latitude > 38 AND dropoff_latitude < 42
"""

In [ ]:
query = rawdata.replace("EVERY_N", "100000")
print(query)
trips = bq.Query(query).execute().result().to_dataframe()
print("Total dataset is {} taxi rides".format(len(trips)))
trips.describe()

In [ ]:
plt.scatter(x = trips["trip_distance"], y = trips["fare_amount"])
plt.title('Taxi Trips')
plt.xlabel("Trip Distance")
plt.ylabel("Fare Amount")
plt.show()

In [ ]:
## the pick-ups
sns.lmplot(x='pickup_longitude', 
           y='pickup_latitude', 
           fit_reg=False, 
           data=trips);
## the drop offs
sns.lmplot(x='dropoff_longitude', 
           y='dropoff_latitude', 
           fit_reg=False, 
           data=trips);

<font color=purple>
    
# Next steps

* Partition the data into Test, Train and Validate
* Explore feature engineering, and feature crossing
* Try models - what variables do we want to predict?